### 필요한 패키지 임포트

In [ ]:
import os
import librosa
import librosa.display
import warnings
warnings.filterwarnings('ignore')
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kurtosis
from scipy.stats import skew
import csv
import time
import datetime

### 파일 경로 불러오기

In [ ]:
# 파일 경로
general = "/content/drive/Othercomputers/내 노트북/Project/Dataset/GTZAN/genres_original"
genre = ('blues classical country disco hiphop jazz metal pop reggae rock').split(" ")

all_audio_path = []

for g in genre:
    audio_path = general + f'/{g}'
    all_audio_path.append(audio_path)

In [ ]:
all_audio_path

['/content/drive/Othercomputers/내 노트북/Project/Dataset/GTZAN/genres_original/blues',
 '/content/drive/Othercomputers/내 노트북/Project/Dataset/GTZAN/genres_original/classical',
 '/content/drive/Othercomputers/내 노트북/Project/Dataset/GTZAN/genres_original/country',
 '/content/drive/Othercomputers/내 노트북/Project/Dataset/GTZAN/genres_original/disco',
 '/content/drive/Othercomputers/내 노트북/Project/Dataset/GTZAN/genres_original/hiphop',
 '/content/drive/Othercomputers/내 노트북/Project/Dataset/GTZAN/genres_original/jazz',
 '/content/drive/Othercomputers/내 노트북/Project/Dataset/GTZAN/genres_original/metal',
 '/content/drive/Othercomputers/내 노트북/Project/Dataset/GTZAN/genres_original/pop',
 '/content/drive/Othercomputers/내 노트북/Project/Dataset/GTZAN/genres_original/reggae',
 '/content/drive/Othercomputers/내 노트북/Project/Dataset/GTZAN/genres_original/rock']

### 함수 정의

In [ ]:
def song_names(audio_path):                      # audio_path : 음원 파일들의 폴더 경로
    files = list(os.listdir(f'{audio_path}'))

    songs = []
    paths = []

    for file in files:
        song = f'{file}'
        path = os.path.join(audio_path, file)
        songs.append(song)
        paths.append(path)

    return songs, paths

In [ ]:
def all_songs_paths(all_audio_path):

    all_song = []
    all_path = []

    for audio_path in all_audio_path:
        songs, paths = song_names(audio_path)
        all_song.append(songs)
        all_path.append(paths)

    return all_song, all_path

In [ ]:
#all_song, all_path = all_songs_paths(all_audio_path)

In [ ]:
#all_song    # 모든 노래 파일 : [[],[],[],...,[]]

In [ ]:
#all_path    # 모든 파일 경로 : [[],[],[],...,[]]

In [ ]:
#all_path[0] # blues 파일 목록

In [ ]:
#for path in all_path:
#    for p in path:
#        print(p)
#        print(p.split("/")[-1])

In [ ]:
# soundwave
def soundwave(all_path):

    audio_files = []
    song_name = []

    for path in all_path:
        for p in path:
            try:
                sn = p.split("/")[-1]
                y, sr = librosa.load(p, duration = 30)
                y, _ = librosa.effects.trim(y)
                song_name.append(sn)
                audio_files.append(y)

            except ValueError as val:
                print(p, val)
            
            except RuntimeError as run:
                print(p, run)

            except IsADirectoryError as iae:
                print(p, iae)

    return audio_files, song_name, sr # sr 디폴트 : 22050 / 변경 가능한 sr 수치 : 44100

In [ ]:
#audio_files, song_name, sr = soundwave(all_path)

In [ ]:
#len(audio_files), len(song_name)

In [ ]:
# Features
def get_features(y, sr):
    # Features to concatenate in the final dictionary
    features = {'centroid': None, 'roloff': None, 'flux': None, 'rmse': None,
                'zcr': None, 'contrast': None, 'bandwidth': None, 'flatness': None, 'chroma_stft': None}
    
    # Count silence
    if 0 < len(y):
        y_sound, _ = librosa.effects.trim(y)
    features['sample_silence'] = len(y) - len(y_sound)

    # Using librosa to calculate the features
    features['chroma_stft']=librosa.feature.chroma_stft(y=y, sr=sr).ravel()
    features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr).ravel()
    features['roloff'] = librosa.feature.spectral_rolloff(y, sr=sr,).ravel()
    features['zcr'] = librosa.feature.zero_crossing_rate(y).ravel()
    features['rmse'] = librosa.feature.rms(y).ravel()
    features['flux'] = librosa.onset.onset_strength(y=y, sr=sr).ravel()
    features['contrast'] = librosa.feature.spectral_contrast(y, sr=sr).ravel()
    features['bandwidth'] = librosa.feature.spectral_bandwidth(y, sr=sr).ravel()
    features['flatness'] = librosa.feature.spectral_flatness(y).ravel()
    
    # harmony, perceptral treatment (+@ 부분)
    harm, perc = librosa.effects.hpss(y)
    features['harm'] = harm.ravel()
    features['perc'] = perc.ravel()

    # MFCC treatment
    mfcc = librosa.feature.mfcc(y, sr=sr)   # n_mfcc=20 (default) 
    for idx, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(idx)] = v_mfcc.ravel()
        
    # Get statistics from the vectors
    def get_moments(descriptors):
        result = {}
        for k, v in descriptors.items():
            result['{}_max'.format(k)] = np.max(v)
            result['{}_min'.format(k)] = np.min(v)
            result['{}_mean'.format(k)] = np.mean(v)
            result['{}_std'.format(k)] = np.std(v)
            result['{}_kurtosis'.format(k)] = kurtosis(v)
            result['{}_skew'.format(k)] = skew(v)
        return result
    
    dict_agg_features = get_moments(features)
#    onset_env = librosa.onset.onset_strength(y=audio_file, sr=sr)
#    dict_agg_features['tempo'] = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
    dict_agg_features['tempo'] = librosa.beat.tempo(y, sr=sr)[0]
    
    return dict_agg_features

In [ ]:
# Extract features 
def final_extraction(dataset_dir=all_audio_path):

    last_features = []

    all_song, all_path = all_songs_paths(all_audio_path)
    audio_files, song_name, sr = soundwave(all_path)

    for audio in audio_files:
        features = get_features(audio, sr)
        last_features.append(features)

    return last_features, song_name

In [ ]:
%%time

last_features, song_name = final_extraction(all_audio_path)

/content/drive/Othercomputers/내 노트북/Project/Dataset/GTZAN/genres_original/jazz/.ipynb_checkpoints [Errno 21] Is a directory: '/content/drive/Othercomputers/내 노트북/Project/Dataset/GTZAN/genres_original/jazz/.ipynb_checkpoints'
CPU times: user 49min 37s, sys: 8min 19s, total: 57min 56s
Wall time: 51min 53s


In [ ]:
len(last_features), len(song_name)

(999, 999)

In [ ]:
def df_all_features(last_features, songs):
    df_features = pd.DataFrame(last_features)
    df_songs = pd.DataFrame(song_name, columns=['song'])

    return pd.concat([df_songs, df_features], axis=1)

In [ ]:
df_all_features = df_all_features(last_features, song_name)
df_all_features.head()

,song,centroid_max,centroid_min,centroid_mean,centroid_std,centroid_kurtosis,centroid_skew,roloff_max,roloff_min,roloff_mean,...,mfcc_18_std,mfcc_18_kurtosis,mfcc_18_skew,mfcc_19_max,mfcc_19_min,mfcc_19_mean,mfcc_19_std,mfcc_19_kurtosis,mfcc_19_skew,tempo
0,blues.00000.wav,4031.364538,1027.079258,1784.420446,360.263361,5.246790,1.537974,7644.287109,1830.322266,3806.485316,...,7.422605,1.236302,0.492951,32.866341,-23.219631,1.219929,6.853561,0.534537,0.258880,123.046875
1,blues.00002.wav,4669.770713,417.254802,1552.481958,395.634070,11.609763,1.930126,6556.860352,441.430664,3040.514948,...,6.828190,0.349044,0.675960,16.524359,-29.785427,-2.226821,5.529573,0.976377,-0.080558,161.499023
2,blues.00003.wav,3236.442116,318.713500,1070.119953,429.532791,1.803354,1.001449,7611.987305,387.597656,2185.028454,...,6.107267,0.175515,-0.124392,16.595015,-21.426250,-3.408233,5.654808,0.880601,0.255001,172.265625
3,blues.00004.wav,4464.156642,849.658285,1835.494603,585.961468,1.536058,1.049734,7084.423828,1152.026367,3580.945013,...,7.932865,0.378629,-0.042242,4.889853,-41.730530,-11.703781,7.432079,0.651141,-0.701559,135.999178
4,blues.00010.wav,3202.187008,436.072310,1410.469096,453.302918,0.489085,0.669760,6707.592773,516.796875,2767.274933,...,6.872131,0.929898,-0.062250,20.802301,-29.235157,-4.986113,6.826524,1.310209,0.176304,161.499023


In [ ]:
# export the data
filename = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
df_all_features.to_csv(f'/content/drive/Othercomputers/내 노트북/Project/Models/Music_Genre_Classification_Recommendation/Feature_Result_csv/{filename}.csv')

In [ ]:
# import the data
df_all_features = pd.read_csv(f'/content/drive/Othercomputers/내 노트북/Project/Models/Music_Genre_Classification_Recommendation/Feature_Result_csv/{filename}.csv', index_col = 0)

In [ ]:
df_all_features

,song,centroid_max,centroid_min,centroid_mean,centroid_std,centroid_kurtosis,centroid_skew,roloff_max,roloff_min,roloff_mean,...,mfcc_18_std,mfcc_18_kurtosis,mfcc_18_skew,mfcc_19_max,mfcc_19_min,mfcc_19_mean,mfcc_19_std,mfcc_19_kurtosis,mfcc_19_skew,tempo
0,blues.00000.wav,4031.364538,1027.079258,1784.420446,360.263361,5.246790,1.537974,7644.287109,1830.322266,3806.485316,...,7.422605,1.236302,0.492951,32.866340,-23.219631,1.219928,6.853561,0.534537,0.258880,123.046875
1,blues.00002.wav,4669.770713,417.254802,1552.481958,395.634070,11.609763,1.930126,6556.860352,441.430664,3040.514948,...,6.828190,0.349044,0.675960,16.524359,-29.785427,-2.226821,5.529573,0.976377,-0.080558,161.499023
2,blues.00003.wav,3236.442116,318.713500,1070.119953,429.532791,1.803354,1.001449,7611.987305,387.597656,2185.028454,...,6.107267,0.175515,-0.124392,16.595015,-21.426250,-3.408233,5.654808,0.880601,0.255001,172.265625
3,blues.00004.wav,4464.156642,849.658285,1835.494603,585.961468,1.536058,1.049734,7084.423828,1152.026367,3580.945013,...,7.932865,0.378629,-0.042242,4.889854,-41.730530,-11.703781,7.432079,0.651141,-0.701559,135.999178
4,blues.00010.wav,3202.187008,436.072310,1410.469096,453.302918,0.489085,0.669760,6707.592773,516.796875,2767.274933,...,6.872131,0.929898,-0.062250,20.802301,-29.235157,-4.986113,6.826524,1.310209,0.176304,161.499023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,rock.00095.wav,4493.780095,928.102055,2008.581132,531.170411,2.519661,1.284807,7902.685547,1894.921875,4254.215942,...,7.408438,0.939795,-0.538604,21.106041,-22.082030,-1.189238,7.068684,0.069182,-0.018390,103.359375
995,rock.00094.wav,3958.563217,1727.080720,2629.346944,306.122150,0.889462,0.539890,7192.089844,3908.276367,5086.735908,...,5.616143,0.401052,0.455727,20.615631,-16.877655,2.251966,6.207781,-0.194352,0.089831,112.347147
996,rock.00091.wav,4564.080112,1243.158267,2474.462256,508.230444,1.316006,0.984008,7773.486328,2670.117188,4754.529583,...,4.864528,0.025915,0.001973,19.045689,-16.979542,-0.845949,4.901772,0.549127,0.175726,143.554688
997,rock.00099.wav,4006.645658,437.156737,1609.442919,649.899970,0.192579,0.581832,7601.220703,322.998047,3246.280370,...,8.498178,0.306571,-0.011078,29.696540,-21.608267,1.158525,7.048834,0.793912,0.589480,123.046875


### 특정값 포함하는 데이터 추출 (데이터 분포 확인용)

In [ ]:
#contains_metal = df_all_features['song'].str.contains("metal")
#subset_df = df_all_features[contains_metal]

In [ ]:
#subset_df['song'].count()